In [4]:
import sympy as sp
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sympy import *
from ipywidgets import interact, FloatSlider, interact_manual


In [5]:
# ------------------------------------------------------------------
# 0.  Pauli-basis helper functions
# ------------------------------------------------------------------
I2 = sp.eye(2)
sx = sp.Matrix([[0, 1], [1, 0]])
sy = sp.Matrix([[0, -sp.I], [sp.I, 0]])
sz = sp.Matrix([[1, 0], [0, -1]])



In [6]:
def coord_to_matrix(w_0, x_0, y_0, z_0):
    """Return x0*I + x1*σx + x2*σy + x3*σz."""
    return w_0*I2 + x_0*sx + y_0*sy + z_0*sz

def f_of_pauli(w_0, x_0, y_0, z_0, func):
    """
    Spectral functional calculus for an Hermitian operator.
    Returns f(A) using eigenprojector formula.
    """
    r  = sp.sqrt(x_0**2 + y_0**2 + z_0**2)
    lam_p, lam_m = w_0 + r, w_0 - r
    a = sp.Rational(1, 2) * (func(lam_p) + func(lam_m))
    b = sp.Rational(1, 2) * (func(lam_p) - func(lam_m))
    # Unit vector  n̂ = x⃗ / r  (SymPy handles r→0 with Piecewise)
    nx, ny, nz = x_0/r, y_0/r, z_0/r
    expr = a*I2 + b*(nx*sx + ny*sy + nz*sz)
    return sp.simplify(expr)          # explicit call



In [7]:
# from density operator in Pauli basis to coordinates

def bloch_coords(rho):
    #rx = sp.simplify(sp.trace(rho * sx))
    #ry = sp.simplify(sp.trace(rho * sy))
    #rz = sp.simplify(sp.trace(rho * sz))
    rx = sp.trace(rho * sx)
    ry = sp.trace(rho * sy)
    rz = sp.trace(rho * sz)
    return [rx, ry, rz]

In [8]:
# ------------------------------------------------------------------
# 1.  Declare symbolic coordinates
# ------------------------------------------------------------------
x_0, y_0, z_0 = sp.symbols('x_0 y_0 z_0', real=True)
a_1, a_2, a_3 = sp.symbols('a_1 a_2 a_3', real=True)
t             = sp.symbols('t', real=True)

rho = coord_to_matrix(1/2, x_0/2, y_0/2, z_0/2)

print(rho)


Matrix([[z_0/2 + 0.5, x_0/2 - I*y_0/2], [x_0/2 + I*y_0/2, 0.5 - z_0/2]])


In [9]:
print("Loading symbolic expressions of geodesic of BKM")

print("Computing ln(rho)")

log_rho   = f_of_pauli(1/2, x_0/2, y_0/2, z_0/2, sp.log)    # log(p)

print("Computing ta")

ta      = coord_to_matrix(0, t*a_1, t*a_2, t*a_3)  # t a

print("Computing ln(rho) +ta")

A       = (log_rho + ta).expand()

print("Computing coordinates of ln(rho) +ta")

# Extract Pauli components of A
A0 = (A.trace() / 2).expand()
A1 = (A[0,1] + A[1,0]) / 2
A2 = (A[1,0] - A[0,1]) / (2*sp.I)
A3 = (A[0,0] - A[1,1]) / 2

print("Computing exp(ln(rho) +ta)")

num1  = f_of_pauli(A0, A1, A2, A3, sp.exp)

print("Computing Tr(exp(ln(rho) +ta))")

den1  = sp.trace(num1)

print("Final expression")

#rho_BKM = (num1 / den1).simplify()

rho_BKM = num1 / den1


print("Extracting coordinates in the Pauli basis")

bloch_BKM = bloch_coords(rho_BKM)

Loading symbolic expressions of geodesic of BKM
Computing ln(rho)
Computing ta
Computing ln(rho) +ta
Computing coordinates of ln(rho) +ta
Computing exp(ln(rho) +ta)
Computing Tr(exp(ln(rho) +ta))
Final expression
Extracting coordinates in the Pauli basis


In [10]:
print("Loading symbolic expressions of geodesic of BH")

print("exponential of ta")

exp_ta = f_of_pauli(0, t*a_1, t*a_2, t*a_3, sp.exp)

print("exp(ta)rho exp(ta)")

numer  = (exp_ta * rho * exp_ta).expand()

print("trace of exp(ta)rho exp(ta)")
denom  = sp.trace(numer)

print("final expression")
#rho_BH  = (numer / denom).simplify()

rho_BH  = numer / denom

print("Extracting coordinates in the Pauli basis")

bloch_BH = bloch_coords(rho_BH)


Loading symbolic expressions of geodesic of BH
exponential of ta
exp(ta)rho exp(ta)
trace of exp(ta)rho exp(ta)
final expression
Extracting coordinates in the Pauli basis


In [11]:
print("Loading symbolic expressions of geodesic of WY")

print("sqrt(rho)")

sqrt_rho   = f_of_pauli(1/2, x_0/2, y_0/2, z_0/2, sp.sqrt)    

 

print("exponential of ta")

exp_ta = f_of_pauli(0, t*a_1, t*a_2, t*a_3, sp.exp)
exp_2ta = f_of_pauli(0, 2*t*a_1, 2*t*a_2, 2*t*a_3, sp.exp)

print("exp(ta) sqrt(rho) exp(2ta) sqrt(rho)")

numer  = (exp_ta * sqrt_rho * exp_2ta * sqrt_rho * exp_ta).expand()

print("trace of exp(ta) sqrt(rho) exp(2ta) sqrt(rho)")
denom  = sp.trace(numer)

print("final expression")


rho_WY  = numer / denom

print("Extracting coordinates in the Pauli basis")

bloch_WY = bloch_coords(rho_WY)


Loading symbolic expressions of geodesic of WY
sqrt(rho)
exponential of ta
exp(ta) sqrt(rho) exp(2ta) sqrt(rho)
trace of exp(ta) sqrt(rho) exp(2ta) sqrt(rho)
final expression
Extracting coordinates in the Pauli basis


In [12]:
print("Loading symbolic expressions of geodesic of CDN")

print("cbrt(rho)")

cbrt_rho   = f_of_pauli(1/2, x_0/2, y_0/2, z_0/2, sp.cbrt)    

 

print("exponential of ta")

exp_ta = f_of_pauli(0, t*a_1, t*a_2, t*a_3, sp.exp)
exp_2ta = f_of_pauli(0, 2*t*a_1, 2*t*a_2, 2*t*a_3, sp.exp)


print("exp(ta) cbrt(rho) exp(2ta) cbrt(rho) exp(2ta) cbrt(rho) exp(ta)")

numer  = (exp_ta * cbrt_rho * exp_2ta * cbrt_rho * exp_2ta * cbrt_rho * exp_ta).expand()

print("trace of exp(ta) cbrt(rho) exp(2ta) cbrt(rho) exp(2ta) cbrt(rho) exp(ta)")
denom  = sp.trace(numer)

print("final expression")


rho_CDN  = numer / denom

print("Extracting coordinates in the Pauli basis")

bloch_CDN = bloch_coords(rho_CDN)

Loading symbolic expressions of geodesic of CDN
cbrt(rho)
exponential of ta
exp(ta) cbrt(rho) exp(2ta) cbrt(rho) exp(2ta) cbrt(rho) exp(ta)
trace of exp(ta) cbrt(rho) exp(2ta) cbrt(rho) exp(2ta) cbrt(rho) exp(ta)
final expression
Extracting coordinates in the Pauli basis


In [13]:

def draw_bloch_sphere(ax, alpha=0.1):
    u, v = np.mgrid[0:2*np.pi:100j, 0:np.pi:50j]
    x = np.cos(u) * np.sin(v)
    y = np.sin(u) * np.sin(v)
    z = np.cos(v)
    ax.plot_surface(x, y, z, color='lightblue', alpha=alpha, linewidth=0)
    
    # Draw axis lines
    ax.plot([-1, 1], [0, 0], [0, 0], color='k', lw=0.5)
    ax.plot([0, 0], [-1, 1], [0, 0], color='k', lw=0.5)
    ax.plot([0, 0], [0, 0], [-1, 1], color='k', lw=0.5)
    
    ax.set_xlim([-1, 1])
    ax.set_ylim([-1, 1])
    ax.set_zlim([-1, 1])
    ax.set_box_aspect([1, 1, 1])
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")
    ax.set_title("Bloch Sphere + Geodesic")


 



def interactive_plot(x0=0, y0=0, z0=0.4, ax=0, ay=0, az=2):
    # Turn off interactive plotting temporarily
    plt.ioff()


    # Pre-create the figure and axes
    fig = plt.figure(figsize=(12, 8))
    ax1 = fig.add_subplot(111, projection='3d')
    draw_bloch_sphere(ax1, alpha=0.15)
    ax1.set_title("Quantum State Evolution on Bloch Sphere")

    fig2 = plt.figure(figsize=(12, 8))
    ax2 = fig2.add_subplot(111)
    ax2.set_xlabel('Time t')
    ax2.set_ylabel('Bloch Coordinates')
    ax2.set_title('Bloch x-Components vs Time')
    ax2.grid(True, alpha=0.3)
    ax2.axhline(y=0, color='k', linestyle='--', alpha=0.3)
    
    fig3 = plt.figure(figsize=(12, 8))
    ax3 = fig3.add_subplot(111)
    ax3.set_xlabel('Time t')
    ax3.set_ylabel('Bloch y-coordinates')
    ax3.set_title('Bloch y-components vs Time')
    ax3.grid(True, alpha=0.3)
    ax3.axhline(y=0, color='k', linestyle='--', alpha=0.3)

    fig4 = plt.figure(figsize=(12, 8))
    ax4 = fig4.add_subplot(111)
    ax4.set_xlabel('Time t')
    ax4.set_ylabel('Bloch z-coordinates')
    ax4.set_title('Bloch z-components vs Time')
    ax4.grid(True, alpha=0.3)
    ax4.axhline(y=0, color='k', linestyle='--', alpha=0.3)

  
 

 
    
    #plt.tight_layout()



    
    if x0**2 + y0**2 + z0**2 > 1:
        print("Invalid Bloch vector: must be inside unit sphere")
        return
    # Create substitution dictionary mapping symbols to values
    subs = {x_0: x0, y_0: y0, z_0: z0, a_1: ax, a_2: ay, a_3: az}
 
 
    

 

    bloch_BH_funcs = [lambdify(t, comp.subs(subs), 'numpy') for comp in bloch_BH]    

    bloch_BKM_funcs = [lambdify(t, comp.subs(subs), 'numpy') for comp in bloch_BKM]    

    bloch_WY_funcs = [lambdify(t, comp.subs(subs), 'numpy') for comp in bloch_WY] 
    
    bloch_CDN_funcs = [lambdify(t, comp.subs(subs), 'numpy') for comp in bloch_CDN] 
        
    
    
    # Compute trajectory
    ts = np.linspace(-3, 3, 1000)
    # choices like x0=y0=a1=a2=0 return an integer for xs and ys that is later incompatible with scatter2, and this is why the if statement below is used
    results_BH = []
    for f in bloch_BH_funcs:
        result_BH = f(ts)
        if np.isscalar(result_BH):
            # It's a constant, so create an array of that constant
            results_BH.append(np.full(len(ts), float(result_BH)))
        else:
            results_BH.append(np.real(result_BH).astype(np.float64))
        
    xsBH, ysBH, zsBH = results_BH
    
    purity_final_state_BH = xsBH[-1]**2 + ysBH[-1]**2 + zsBH[-1]**2
    # the :.numf below determines the precision of the approximation  
    print(f"Final state coordinates BH: X_BH~{xsBH[-1]:}, Y_BH~{ysBH[-1]:}, Z_BH~{zsBH[-1]:} \nFinal state purity BH: {1/2*(1+ xsBH[-1]**2 + ysBH[-1]**2 + zsBH[-1]**2):}") 

    results_BKM = []
    for f in bloch_BKM_funcs:
        result_BKM = f(ts)
        if np.isscalar(result_BKM):
            # It's a constant, so create an array of that constant
            results_BKM.append(np.full(len(ts), float(result_BKM)))
        else:
            results_BKM.append(np.real(result_BKM).astype(np.float64))
        
    xsBKM, ysBKM, zsBKM = results_BKM
    
    purity_final_state_BKM = xsBKM[-1]**2 + ysBKM[-1]**2 + zsBKM[-1]**2
    # the :.numf below determines the precision of the approximation  
    print(f"Final state coordinates BKM: X_BKM~{xsBKM[-1]:}, Y_BKM~{ysBKM[-1]:}, Z_BKM~{zsBH[-1]:} \nFinal state purity BKM: {1/2*(1+ xsBKM[-1]**2 + ysBKM[-1]**2 + zsBKM[-1]**2):}") 
    
    results_WY = []
    for f in bloch_WY_funcs:
        result_WY = f(ts)
        if np.isscalar(result_WY):
            # It's a constant, so create an array of that constant
            results_WY.append(np.full(len(ts), float(result_WY)))
        else:
            results_WY.append(np.real(result_WY).astype(np.float64))
        
    xsWY, ysWY, zsWY = results_WY
    
    purity_final_state_WY = xsWY[-1]**2 + ysWY[-1]**2 + zsWY[-1]**2
    # the :.numf below determines the precision of the approximation  
    print(f"Final state coordinates WY: X_WY~{xsWY[-1]:}, Y_WY~{ysBKM[-1]:}, Z_WY~{zsBH[-1]:} \nFinal state purity WY: {1/2*(1+ xsWY[-1]**2 + ysWY[-1]**2 + zsWY[-1]**2):}") 
    
    results_CDN = []
    for f in bloch_CDN_funcs:
        result_CDN = f(ts)
        if np.isscalar(result_CDN):
            # It's a constant, so create an array of that constant
            results_CDN.append(np.full(len(ts), float(result_CDN)))
        else:
            results_CDN.append(np.real(result_CDN).astype(np.float64))
        
    xsCDN, ysCDN, zsCDN = results_CDN
    
    purity_final_state_CDN = xsCDN[-1]**2 + ysCDN[-1]**2 + zsCDN[-1]**2
    # the :.numf below determines the precision of the approximation  
    print(f"Final state coordinates CDN: X_CDN~{xsCDN[-1]:}, Y_CDN~{ysCDN[-1]:}, Z_CDN~{zsCDN[-1]:} \nFinal state purity CDN: {1/2*(1+ xsCDN[-1]**2 + ysCDN[-1]**2 + zsCDN[-1]**2):}") 

    
    # Plot trajectory BH
    line = ax1.plot(xsBH, ysBH, zsBH, color='red', linewidth=2, label='Geodesic BH', alpha=0.8)
    line = ax1.plot(xsBKM, ysBKM, zsBKM, color='blue', linewidth=2, label='Geodesic BKM', alpha=0.8)
    line = ax1.plot(xsWY, ysWY, zsWY, color='green', linewidth=2, label='Geodesic WY', alpha=0.8)
    line = ax1.plot(xsCDN, ysCDN, zsCDN, color='yellow', linewidth=2, label='Geodesic CDN', alpha=0.8)
 
    
    # Mark initial and final states
    scatter1 = ax1.scatter([x0], [y0], [z0], color='black', s=100, label='Initial state', alpha=0.9)
    scatter2 = ax1.scatter([xsBH[-1]], [ysBH[-1]], [zsBH[-1]], color='red', s=80, label='Final state BH', alpha=0.9)
    scatter2 = ax1.scatter([xsBKM[-1]], [ysBKM[-1]], [zsBKM[-1]], color='blue', s=80, label='Final state BKM', alpha=0.9)
    scatter2 = ax1.scatter([xsWY[-1]], [ysWY[-1]], [zsWY[-1]], color='green', s=80, label='Final state WY', alpha=0.9)
    scatter2 = ax1.scatter([xsCDN[-1]], [ysCDN[-1]], [zsCDN[-1]], color='yellow', s=80, label='Final state CDN', alpha=0.9)

    
    # Add legend for 3D plot
    ax1.legend(loc='upper right')
    
    # 2D trajectory components plot
    line2d_x = ax2.plot(ts, xsBH, 'r-', label='X_BH(t)', alpha=0.8)
    line2d_x = ax2.plot(ts, xsBKM, 'b-', label='X_BKM(t)', alpha=0.8)
    line2d_x = ax2.plot(ts, xsWY, 'g-', label='X_WY(t)', alpha=0.8)
    line2d_x = ax2.plot(ts, xsCDN, 'y-', label='X_CDN(t)', alpha=0.8)

    line2d_y = ax3.plot(ts, ysBH, 'r-', label='Y_BH(t)', alpha=0.8)
    line2d_y = ax3.plot(ts, ysBKM, 'b-', label='Y_BKM(t)', alpha=0.8)
    line2d_y = ax3.plot(ts, ysWY, 'g-', label='Y_WY(t)', alpha=0.8)
    line2d_y = ax3.plot(ts, ysCDN, 'y-', label='Y_CDN(t)', alpha=0.8)
    
    line2d_z = ax4.plot(ts, zsBH, 'r-', label='Z_BH(t)', alpha=0.8)
    line2d_z = ax4.plot(ts, zsBKM, 'b-', label='Z_BKM(t)', alpha=0.8)
    line2d_z = ax4.plot(ts, zsWY, 'g-', label='Z_WY(t)', alpha=0.8)
    line2d_z = ax4.plot(ts, zsCDN, 'y-', label='Z_CDN(t)', alpha=0.8)
    
    
    
    # Add legend for 2D plot
    ax2.legend(loc='upper right')
    ax3.legend(loc='upper right')
    ax4.legend(loc='upper right')
 
    

    
    plt.tight_layout()
    plt.ion()  # Turn interactive plotting back on
    plt.show()
 

 

interact_manual(
    interactive_plot,
    x0=FloatSlider(min=-1.0, max=1.0, step=0.05, value=0),
    y0=FloatSlider(min=-1.0, max=1.0, step=0.05, value=0),
    z0=FloatSlider(min=-1.0, max=1.0, step=0.05, value=0.4),
    ax=FloatSlider(min=-10, max=10, step=0.5, value=0),
    ay=FloatSlider(min=-10, max=10, step=0.5, value=0),
    az=FloatSlider(min=-10, max=10, step=0.5, value=2)
); #semicolon is to suppress <function __main__.interactive_plot(x0=0, y0=0, z0=0.4, a_1=0, a_2=0, a_3=2)> to appear at the end of the cell

  

interactive(children=(FloatSlider(value=0.0, description='x0', max=1.0, min=-1.0, step=0.05), FloatSlider(valu…